In [ ]:
import os
import requests
import gradio as gr
import google.generativeai as genai
from dotenv import load_dotenv
from ultralytics import YOLO
import cv2

# Load environment variables
load_dotenv()
genai.configure(api_key=os.getenv("GEMINI_API_KEY"))

# YOLO model setup
model = YOLO("best.pt")  # Replace with your model path if needed

# Google Gemini Model configuration
generation_config = {
    "temperature": 0,
    "top_p": 0.95,
    "top_k": 64,
    "max_output_tokens": 8192,
    "response_mime_type": "text/plain",
}

safety_settings = [
    {"category": "HARM_CATEGORY_HARASSMENT", "threshold": "BLOCK_NONE"},
    {"category": "HARM_CATEGORY_HATE_SPEECH", "threshold": "BLOCK_MEDIUM_AND_ABOVE"},
    {"category": "HARM_CATEGORY_SEXUALLY_EXPLICIT", "threshold": "BLOCK_MEDIUM_AND_ABOVE"},
    {"category": "HARM_CATEGORY_DANGEROUS_CONTENT", "threshold": "BLOCK_MEDIUM_AND_ABOVE"},
]

system_instruction = """
I want a food recipe based on the following ingredients: {}.
Please format the recipe with the following sections:
Ingredients: List all ingredients required for the recipe.
Instructions: Provide clear step-by-step instructions for preparing the recipe.
Optional Tips: Include any optional tips for variation or improvement of the recipe.
"""

# Updated recipe generation part in process_image
def process_image(image):
    # Run object detection on the uploaded image
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    results = model.predict(source=image, conf=0.35, save=True)
    
    # Collect detected items
    detected_items = {}
    for result in results:
        for box in result.boxes:
            cls = box.cls
            class_name = model.names[int(cls)]
            detected_items[class_name] = detected_items.get(class_name, 0) + 1
    
    # Prepare the list of detected items for the AI model
    ingredients_list = ", ".join(detected_items.keys())
    prompt = system_instruction.format(ingredients_list)
    
    # Generate a recipe based on detected items
    modelresponse = genai.GenerativeModel(
        model_name="gemini-1.5-pro",
        safety_settings=safety_settings,
        generation_config=generation_config,
        system_instruction="I want a food recipe based on the following ingredients:{}. Please format the recipe with the following sections:Ingredients: List all ingredients required for the recipe.Instructions: Provide clear step-by-step instructions for preparing the recipe.Optional Tips: Include any optional tips for variation or improvement of the recipe.",
    )
    chat_session = modelresponse.start_chat(
    history=[]
    )

    print()
    print("Planning a menu based on the ingredients list...")

    user_input = (str(detected_items))
    print()

    response = chat_session.send_message(user_input)

    # Extract response text
    recipe_response = response.text if hasattr(response, "text") else str(response)
    
    # Return detected items and recipe
    return detected_items, recipe_response


# Set up the Gradio interface
iface = gr.Interface(
    fn=process_image,
    inputs="image",
    outputs=["json", "text"],
    title="Object Detection and Recipe Generator",
    description="Upload a picture of food, and this app will detect ingredients and generate a recipe for you."
)

# Launch the Gradio app
iface.launch()


* Running on local URL:  http://127.0.0.1:7860

To create a public link, set `share=True` in `launch()`.


Traceback (most recent call last):
  File "c:\Users\jedsa\anaconda3\Lib\site-packages\gradio\queueing.py", line 624, in process_events
    response = await route_utils.call_process_api(
               ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\jedsa\anaconda3\Lib\site-packages\gradio\route_utils.py", line 323, in call_process_api
    output = await app.get_blocks().process_api(
             ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\jedsa\anaconda3\Lib\site-packages\gradio\blocks.py", line 2018, in process_api
    result = await self.call_function(
             ^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\jedsa\anaconda3\Lib\site-packages\gradio\blocks.py", line 1567, in call_function
    prediction = await anyio.to_thread.run_sync(  # type: ignore
                 ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\jedsa\anaconda3\Lib\site-packages\anyio\to_thread.py", line 56, in run_sync
    return await get_async_backend().run_sync_in_worker_thread(
 


0: 640x640 1 eggplant, 1 garlic, 1 onion, 280.7ms
Speed: 14.0ms preprocess, 280.7ms inference, 17.0ms postprocess per image at shape (1, 3, 640, 640)
Results saved to runs\detect\predict2

Planning a menu based on the ingredients list...


0: 640x640 1 bell_pepper, 1 garlic, 3 lemons, 2 onions, 1 pork, 1 potato, 2 tomaos, 307.3ms
Speed: 8.5ms preprocess, 307.3ms inference, 47.4ms postprocess per image at shape (1, 3, 640, 640)
Results saved to runs\detect\predict2

Planning a menu based on the ingredients list...

